# Analysis of fragmented malware detection by Suricata

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import orjson

In [36]:
def get_json_data(file):
    return [orjson.loads(line) for line in open("suricata_logs/All.ElectroRAT/eve.json", "rb")]

In [37]:
def get_alerts(json_data):
    alerts = [line for line in json_data if "alert" in line]
    return alerts

In [ ]:
file = "suricata_logs/All.ElectroRAT/eve.json"
json_data = get_json_data(file)
alerts = get_alerts(json_data)